# Data Pipeline & EDA: Mining Logistics Optimization

**Role:** ML Engineer A  
**Project:** Mining Value Chain - Logistics Simulation  
**Tujuan:** Menyiapkan data operasional untuk pemodelan prediksi *Cycle Time* dan *Risiko Delay*.

---

### Deskripsi Tugas

Notebook ini adalah tahap pertama (Notebook 1 dari 3) dalam pipeline Machine Learning. Fokus utama kita adalah:

1. **Data Ingestion:** Memuat dataset operasional infrastruktur tambang (`ms_infra_transport_cycle_10k.csv`).
2. **Data Splitting:** Memisahkan data menjadi dua aliran: dataset operasional (untuk Regresi) dan dataset lengkap termasuk breakdown (untuk Klasifikasi Risiko)
3. **Data Saving:** Menyimpan data bersih dalam format `csv` untuk efisiensi di tahap modeling.

## 1. Setup Environment

Mengimpor library standar untuk manipulasi data tabular (`pandas`) dan visualisasi data (`matplotlib`, `seaborn`).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Data Ingestion (Pemuatan Data)

Kita memuat dataset mentah. Langkah krusial di sini adalah memastikan kolom tanggal (`tanggal_operasi`) dibaca sebagai tipe data `datetime` agar urutan waktu (time-series) terjaga.

In [2]:
# Load dataset
nama_file = "../../data/processed/infra_transport_cycle_10k.csv"

try:
    # Baca file CSV
    df = pd.read_csv(nama_file)
    print(f"Berhasil memuat {nama_file}")
    
    # Konversi kolom tanggal ke tipe datetime
    df['tanggal_operasi'] = pd.to_datetime(df['tanggal_operasi'])
    
    # Urutkan data berdasarkan waktu (penting untuk time-series)
    df = df.sort_values('tanggal_operasi')
    
    print("Tipe data tanggal berhasil diperbaiki")
    
except FileNotFoundError:
    print(f"ERROR: File {nama_file} tidak ditemukan")
    print("Pastikan file csv berada di folder yang sama dengan notebook ini.")

Berhasil memuat ../../data/processed/infra_transport_cycle_10k.csv
Tipe data tanggal berhasil diperbaiki


## 3. Inspeksi Kesehatan Data (Data Health Check)

Sebelum pemrosesan, kita melakukan *screening* awal untuk mendeteksi anomali:

* **Tipe Data:** Memastikan angka dibaca sebagai numerik, bukan teks.
* **Missing Values:** Mengidentifikasi kolom yang memiliki data kosong (Null/NaN).
* **Statistik Deskriptif:** Melihat sebaran data target (`cycle_time_avg_jam`).

In [3]:
# Cek informasi dataset
print("=" * 50)
print("INFO DATASET")
print("=" * 50)
df.info()

# Cek missing values
print("\n" + "=" * 50)
print("MISSING VALUES")
print("=" * 50)
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])  # Tampilkan hanya kolom dengan missing values

# Statistik deskriptif target variable
print("\n" + "=" * 50)
print("STATISTIK CYCLE TIME")
print("=" * 50)
print(df['cycle_time_avg_jam'].describe())

INFO DATASET
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 8791
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id_record               10000 non-null  object        
 1   id_alat                 10000 non-null  object        
 2   tanggal_operasi         10000 non-null  datetime64[ns]
 3   shift                   10000 non-null  object        
 4   jam_mulai               10000 non-null  object        
 5   jam_selesai             10000 non-null  object        
 6   status_operasi          10000 non-null  object        
 7   durasi_jam              10000 non-null  float64       
 8   material_dipindah       10000 non-null  object        
 9   total_muatan_ton        10000 non-null  float64       
 10  jumlah_ritase           10000 non-null  int64         
 11  id_operator             10000 non-null  object        
 12  lokasi_kode             10000 non-null 

## 4. Pembuatan Fitur Baru dan Split Logic

Terdapat fitur-fitur yang akan menyebabkan data leakage dan jika dihapus, model tidak bisa melihat polanya sehingga pembuatan fitur baru ini penting untuk model melihat pola.

In [4]:
# LAG FEATURES (Fitur Antrian)
# Urutkan data berdasarkan lokasi dan waktu
df = df.sort_values(by=['lokasi_kode', 'tanggal_operasi'])

# Fitur 1: Cycle time unit sebelumnya di lokasi yang sama
# Logika: Jika truk depan lambat, truk belakang akan ikut lambat (efek antrian)
df['prev_cycle_time'] = df.groupby('lokasi_kode')['cycle_time_avg_jam'].shift(1)

# Fitur 2: Rolling average dari 3 cycle terakhir
# Logika: Rata-rata performa 3 truk terakhir di lokasi tersebut
df['avg_last_3_cycles'] = df.groupby('lokasi_kode')['cycle_time_avg_jam'].transform(
    lambda x: x.rolling(window=3).mean()
)

# Isi nilai NaN dengan backward fill (untuk data awal yang tidak punya history)
df = df.bfill()

print("Fitur lag berhasil ditambahkan!")

Fitur lag berhasil ditambahkan!


In [5]:
# SPLIT LOGIC - Memisahkan dataset untuk Regresi dan Klasifikasi

# A. Dataset untuk REGRESI (Prediksi Cycle Time)
# Syarat: Unit harus dalam status 'Beroperasi' dan Cycle Time valid (> 0)
# Data Standby/Breakdown tidak masuk karena cycle time = 0 (tidak logis untuk regresi)
df_reg = df[
    (df['status_operasi'] == 'Beroperasi') &
    (df['cycle_time_avg_jam'] > 0) &
    (df['cycle_time_avg_jam'].notnull())
].copy()

# B. Dataset untuk KLASIFIKASI (Prediksi Risiko Delay/Breakdown)
# Pakai semua data, termasuk Standby/Rusak sebagai contoh kelas "Berisiko"
df_clf = df.copy()

print(f"Dataset Regresi: {len(df_reg)} baris")
print(f"Dataset Klasifikasi: {len(df_clf)} baris")

Dataset Regresi: 7427 baris
Dataset Klasifikasi: 10000 baris


## 5. Data Cleaning and Engineering

**Strategi:** Ditemukan adanya kolom yang tidak penting dan menyebabkan leakage sehingga harus dihapus. Selain itu, ekstraksi jam dan logika hujan sangat berpengaruh.

**Tindakan:**
* Ekstraksi waktu (jam operasi)
* Thresholding logika hujan
* Hapus data leakage
* Handling identifiers

**Alasan:** Waktu sangat berpengaruh untuk traffic tambang, menerapkan logika hujan untuk mengidentifikasi hujan lebat atau tidak, data leakage akan menyebabkan model gagal memahami pola, dan identifiers khususnya di sini (`id_record`) tidak berpengaruh.

In [6]:
def clean_and_engineer(dataframe):
    """
    Membersihkan dan membuat fitur baru pada dataframe.
    
    Parameters:
    -----------
    dataframe : pd.DataFrame
        Dataset yang akan dibersihkan
    
    Returns:
    --------
    pd.DataFrame
        Dataset yang sudah dibersihkan
    """
    # 1. Ekstraksi Waktu
    # Jam operasi sangat mempengaruhi traffic tambang (shift pagi vs malam)
    # Format jam_mulai 'HH:MM:SS', ambil HH saja
    dataframe['jam_operasi'] = dataframe['jam_mulai'].str[:2].astype(int)
    
    # 2. Logika Hujan (Thresholding)
    # Hujan > 10mm = jalan licin, risiko delay tinggi
    dataframe['hujan_deras'] = dataframe['hujan_mm'].apply(lambda x: 1 if x > 10 else 0)
    
    # 3. Hapus Data Leakage
    # Kolom ini adalah hasil perhitungan dari cycle_time, tidak boleh jadi input model
    leakage_cols = ['ritase_per_jam', 'ton_per_jam', 'jumlah_ritase', 
                    'total_muatan_ton', 'durasi_jam']
    dataframe = dataframe.drop(columns=leakage_cols, errors='ignore')
    
    # 4. Handling Identifiers
    # Buang id_record (unik per baris, tidak ada pola)
    # Simpan id_alat dan id_operator (akan di-encode di notebook berikutnya)
    dataframe = dataframe.drop(columns=['id_record'], errors='ignore')
    
    return dataframe

In [7]:
# Terapkan cleaning ke kedua dataset
df_reg_clean = clean_and_engineer(df_reg)
df_clf_clean = clean_and_engineer(df_clf)

# Tampilkan hasil
print(f"\nData Regresi (Unit Beroperasi): {len(df_reg_clean)} baris")
print(f"Data Klasifikasi (Semua Unit): {len(df_clf_clean)} baris")
print(f"Jumlah kolom: {len(df_reg_clean.columns)}")


Data Regresi (Unit Beroperasi): 7427 baris
Data Klasifikasi (Semua Unit): 10000 baris
Jumlah kolom: 30


## 6. Penyimpanan Data (Data Saving)

Menyimpan data yang sudah bersih (Clean Dataset) ke dalam format `.csv`.

---

**Next Step:**
* `data_regression_clean.csv` akan digunakan pada **Notebook 02 (Modeling Regresi)**
* `data_classification_clean.csv` akan digunakan pada **Notebook 03 (Modeling Klasifikasi)**

In [8]:
# Simpan dataset yang sudah dibersihkan
df_reg_clean.to_csv("../../data/processed/data_regression_clean.csv", index=False)
df_clf_clean.to_csv("../../data/processed/data_classification_clean.csv", index=False)

print("\n" + "="*50)
print("DATA BERHASIL DISIMPAN")
print("="*50)
print("1. data_regression_clean.csv -> Untuk Notebook 02 (Modeling Regresi)")
print("2. data_classification_clean.csv -> Untuk Notebook 03 (Modeling Klasifikasi)")
print("\nNotebook 01 selesai!")


DATA BERHASIL DISIMPAN
1. data_regression_clean.csv -> Untuk Notebook 02 (Modeling Regresi)
2. data_classification_clean.csv -> Untuk Notebook 03 (Modeling Klasifikasi)

Notebook 01 selesai!
